In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import pandas as pd
import numpy as np
import os
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from datetime import datetime

In [2]:
os.getcwd()

'/Users/korniltsevdmitry/Desktop/python/projects/recommendation_systems/notebooks'

In [3]:
data = pd.read_csv('../data/result_df.csv', nrows=2000)
# data = pd.read_csv('../data/result_df.csv')

In [4]:
data.head(2)

,hash_inn,okved2,region,0_count_kt,1_count_kt,10_count_kt,1000_count_kt,11_count_kt,12_count_kt,13_count_kt,...,share_72_dt_dt,share_73_dt_dt,share_74_dt_dt,share_75_dt_dt,share_76_dt_dt,share_77_dt_dt,share_78_dt_dt,share_79_dt_dt,share_8_dt_dt,share_9_dt_dt
0,61058,34,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8311,18,86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### add text data to create word2vec embeddings

In [5]:
corpus = [
    'he is a king',
    'she is a queen',
    'he is a man',
    'she is a woman',
    'warsaw is poland capital',
    'berlin is germany capital',
    'paris is france capital',
]

In [14]:
len(corpus)

7

In [6]:
def tokenize_corpus(corpus):
    tokens = [x.split() for x in corpus]
    return tokens

tokenized_corpus = tokenize_corpus(corpus)

In [9]:
tokenized_corpus

[['he', 'is', 'a', 'king'],
 ['she', 'is', 'a', 'queen'],
 ['he', 'is', 'a', 'man'],
 ['she', 'is', 'a', 'woman'],
 ['warsaw', 'is', 'poland', 'capital'],
 ['berlin', 'is', 'germany', 'capital'],
 ['paris', 'is', 'france', 'capital']]

In [21]:
text_col = pd.DataFrame(corpus*300).iloc[0:200,]
text_col.columns = ['text']

In [25]:
text_col['tokens'] = text_col['text'].apply(lambda x: tokenize_corpus(x))

In [26]:
text_col.head(2)

,text,tokens
0,he is a king,"[[h], [e], [], [i], [s], [], [a], [], [k], [i]..."
1,she is a queen,"[[s], [h], [e], [], [i], [s], [], [a], [], [q]..."


In [7]:
vocabulary = []
for sentence in tokenized_corpus:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [8]:
window_size = 2
idx_pairs = []
# for each sentence
for sentence in tokenized_corpus:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

### determ categorical and continious columns

In [10]:
cat_col = ["region"]
cont_col = [col for col in data.columns if 'share' in col]
target_col = ["okved2"]

In [52]:
data = data[cont_col + cat_col + target_col]
data.fillna(0, inplace=True)

In [53]:
len(data.okved2.unique())

75

In [54]:
data.head(2)

,share_0_kt,share_1_kt,share_10_kt,share_1000_kt,share_11_kt,share_12_kt,share_13_kt,share_14_kt,share_15_kt,share_16_kt,...,share_74_dt_dt,share_75_dt_dt,share_76_dt_dt,share_77_dt_dt,share_78_dt_dt,share_79_dt_dt,share_8_dt_dt,share_9_dt_dt,region,okved2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86,34
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86,18


In [55]:
label_encoders = {}
for col in cat_col:
    label_encoders[col] = LabelEncoder()
    data[col] = label_encoders[col].fit_transform(data[col]) 
    data[col] = data[col].astype('category')

In [56]:
Y = LabelEncoder().fit_transform(data[target_col])

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [57]:
X = data[cont_col + cat_col]

In [58]:
emb_cols_dict = {n: len(col.cat.categories) for n, col in X[cat_col].items() if len(col.cat.categories) > 2}
emb_cols = emb_cols_dict.keys()
emb_sizes = [(c, min(50, (c+1)//2)) for _,c in emb_cols_dict.items()]

In [59]:
emb_sizes

[(77, 39)]

In [60]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)
X_train.head(2)

,share_0_kt,share_1_kt,share_10_kt,share_1000_kt,share_11_kt,share_12_kt,share_13_kt,share_14_kt,share_15_kt,share_16_kt,...,share_73_dt_dt,share_74_dt_dt,share_75_dt_dt,share_76_dt_dt,share_77_dt_dt,share_78_dt_dt,share_79_dt_dt,share_8_dt_dt,share_9_dt_dt,region
81,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71
915,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71


In [61]:
class ShelterOutcomeDataset(Dataset):
    def __init__(self, X, Y, emb_cols):
        X = X.copy()
        #categorical columns
        self.X1 = X.loc[:, emb_cols].copy().values.astype(np.int64)
        #numerical columns
        self.X2 = X.drop(columns=emb_cols).copy().values.astype(np.float32)
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.y[idx]

In [62]:
#creating train and valid datasets
train_ds = ShelterOutcomeDataset(X_train, y_train, emb_cols)
valid_ds = ShelterOutcomeDataset(X_val, y_val, emb_cols)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [70]:
class ShelterOutcomeModel(nn.Module):
    def __init__(self, embedding_sizes, n_cont, target_dim):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])
        n_emb = sum(e.embedding_dim for e in self.embeddings) #length of all embeddings combined
        self.n_emb = n_emb
        self.n_cont = n_cont
        self.target_dim = target_dim
        self.lin1 = nn.Linear(self.n_emb + self.n_cont, 800)
        self.lin2 = nn.Linear(800, 280)
        self.lin3 = nn.Linear(280, self.target_dim)
        self.bn1 = nn.BatchNorm1d(self.n_cont)
        self.bn2 = nn.BatchNorm1d(800)
        self.bn3 = nn.BatchNorm1d(280)
        self.emb_drop = nn.Dropout(0.6)
        self.drops = nn.Dropout(0.3)
        

    def forward(self, x_cat, x_cont):
        x = [e(x_cat[:,i]) for i,e in enumerate(self.embeddings)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn1(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.lin1(x))
        x = self.drops(x)
        x = self.bn2(x)
        x = F.relu(self.lin2(x))
        x = self.drops(x)
        x = self.bn3(x)
        x = self.lin3(x)
        return x

In [64]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [35]:
len(cont_col)

162

In [68]:
len(pd.Series(Y).unique())

75

In [71]:
model = ShelterOutcomeModel(embedding_sizes=emb_sizes, n_cont=len(cont_col), target_dim = len(pd.Series(Y).unique()))
to_device(model, device)

ShelterOutcomeModel(
  (embeddings): ModuleList(
    (0): Embedding(77, 39)
  )
  (lin1): Linear(in_features=201, out_features=800, bias=True)
  (lin2): Linear(in_features=800, out_features=280, bias=True)
  (lin3): Linear(in_features=280, out_features=75, bias=True)
  (bn1): BatchNorm1d(162, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (emb_drop): Dropout(p=0.6, inplace=False)
  (drops): Dropout(p=0.3, inplace=False)
)

In [76]:
def get_optimizer(model, lr = 0.001, wd = 0.0):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optim = torch_optim.Adam(parameters, lr=lr, weight_decay=wd)
    return optim

In [89]:
def train_model(model, optim, train_dl):
    model.train()
    total = 0
    sum_loss = 0
    for x1, x2, y in train_dl:
        batch = y.shape[0]
        output = model(x1, x2)
        loss = F.cross_entropy(output, y)   
        optim.zero_grad()
        loss.backward()
        optim.step()
        total += batch
        sum_loss += batch*(loss.item())
        
        print('weights')
        print(torch.sum(model.lin3.weight.data))
        trained_weights = model.lin3.weight.data
#     print(trained_weights)
    return sum_loss/total, trained_weights

In [90]:
# import copy
# init_weights = copy.deepcopy(model.fc1.weight.data)
# for epoch in range(1, 3):
#     for batch_idx, (dat, target) in enumerate(train_loader):
#         data, target = Variable(dat), Variable(target).detach()
#         optimizer.zero_grad()
#         output = model(data)
#         loss = criterion(output, target)
#         loss.backward()
#         optimizer.step()
#         print(torch.sum(model.fc1.weight.data))
# trained_weights = model.fc1.weight.data

In [ ]:
# m = nn.Embedding(...)
# # m.weight contains the embedding weights.
# parameters = m.weight.numpy()
# # save it as you wish

In [91]:
def val_loss(model, valid_dl):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0
    predictions, actuals = list(), list()
    for x1, x2, y in valid_dl:
        current_batch_size = y.shape[0]
        out = model(x1, x2)
        loss = F.cross_entropy(out, y)
        sum_loss += current_batch_size*(loss.item())
        total += current_batch_size
        pred = torch.max(out, 1)[1]
        correct += (pred == y).float().sum().item()
        
        y = y.reshape((len(y), 1))
        pred = pred.reshape((len(pred), 1))
        # store
        predictions.append(pred)
        actuals.append(y)
    print(len(predictions), len(actuals))
    print("valid loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
    predictions, actuals = vstack(predictions), vstack(actuals)
    return actuals, predictions

In [86]:
def train_loop(model, epochs, lr=0.01, wd=0.0):
    optim = get_optimizer(model, lr = lr, wd = wd)
    for i in range(epochs): 
        loss, weights = train_model(model, optim, train_dl)
        print("training loss: ", loss)
        actuals, predictions = val_loss(model, valid_dl)
    return actuals, predictions

In [87]:
batch_size=256
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=True)

In [92]:
actuals, predictions = train_loop(model, epochs=8, lr=0.05, wd=0.00001)
print(classification_report(actuals, predictions))

weights
tensor(-4138.0127)
weights
tensor(-4238.5894)
weights
tensor(-4312.7754)
weights
tensor(-4382.9590)
weights
tensor(-4443.7563)
weights
tensor(-4504.9312)
training loss:  2.0992072959444417
3 3
valid loss 5.630 and accuracy 0.303
weights
tensor(-4551.7847)
weights
tensor(-4592.7144)
weights
tensor(-4632.6152)
weights
tensor(-4671.5112)
weights
tensor(-4713.1274)
weights
tensor(-4760.2236)
training loss:  1.9940433651653688
3 3
valid loss 5.804 and accuracy 0.238
weights
tensor(-4804.0005)
weights
tensor(-4848.6763)
weights
tensor(-4890.6982)
weights
tensor(-4927.4727)
weights
tensor(-4959.8823)
weights
tensor(-4994.0210)
training loss:  1.8931541371701368
3 3
valid loss 5.722 and accuracy 0.241
weights
tensor(-5029.1479)
weights
tensor(-5067.5918)
weights
tensor(-5106.9971)
weights
tensor(-5144.8721)
weights
tensor(-5180.1641)
weights
tensor(-5216.9868)
training loss:  1.7992042972080744
3 3
valid loss 5.742 and accuracy 0.282
weights
tensor(-5251.4819)
weights
tensor(-5283.0698

In [43]:
actuals, predictions = train_loop(model, epochs=8, lr=0.05, wd=0.00001)
print(classification_report(actuals, predictions))

training loss:  2.7150109985951065
310 310
valid loss 2.628 and accuracy 0.331
training loss:  2.6515707370202426
310 310
valid loss 2.628 and accuracy 0.318
training loss:  2.6611339436356714
310 310
valid loss 2.630 and accuracy 0.334
training loss:  2.6687577738265693
310 310
valid loss 2.613 and accuracy 0.335
training loss:  2.672064993795145
310 310
valid loss 2.655 and accuracy 0.329
training loss:  2.679746832163435
310 310
valid loss 2.642 and accuracy 0.332
training loss:  2.67826179908855
310 310
valid loss 2.676 and accuracy 0.304
training loss:  2.67923294631206
310 310
valid loss 2.640 and accuracy 0.310
              precision    recall  f1-score   support

           0       0.18      0.07      0.10       262
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00        29
           3       0.00      0.00      0.00       265
           4       0.25      0.01      0.01      2552
           5       0.00      0.00      0.00      

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
